# Ejemplos GeoPandas C3a

## Carga de librerías y datos

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
!unzip data.zip

## Ejemplo 1: graficar en un mapa los distritos censales (o las partes de ellos) ubicados en el área urbana de la Región Metropolitana, utilizando una escala de colores basada en el costo monetario en pesos que tendría el edificarlos completamente con viviendas de 1 piso.

### Cargamos los archivos .shp correspondientes (shapefiles), utilizando la función `read_file` de GeoPandas

In [ ]:
distritos = gpd.read_file('data/Distritos Censales/Distritos Censales RM.shp')
chile_urbano = gpd.read_file('data/Areas Urbanas/areas_urbanas.shp')

### Revisamos los datos

In [ ]:
distritos.head()

In [ ]:
distritos['NOM_PROV'].value_counts()

In [ ]:
distritos.describe()

In [ ]:
chile_urbano.head()

In [ ]:
chile_urbano.TIPO_AREA.value_counts()

### Graficamos los GeoDataFrame para visualizar los datos

In [ ]:
distritos.plot(figsize=(10,10))


In [ ]:
santiago_urbano = chile_urbano.loc[chile_urbano['NOMBRE']=='Santiago']
santiago_urbano.plot(figsize = (10,10))

### Combinamos los GeoDataFrame

Antes de combinar ambos archivos, tenemos que revisar sus proyecciones

In [ ]:
distritos.crs

In [ ]:
santiago_urbano.crs

En este caso ambos tienen la misma proyeccion, por lo tanto creamos los distritos urbanos (con la intersección)

In [ ]:
distritos_urbanos = gpd.overlay(santiago_urbano, distritos, how='intersection')

In [ ]:
distritos_urbanos.plot(figsize=(10,10))

### Dado que ya obtuvimos los distritos urbanos, podemos construir la informacion solicitada

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
distritos_urbanos.head(10)

Al revisar los datos como tabla, vemos que se tiene el precio del metro cuadrado en UF. Como necesitamos el valor en pesos, multiplicamos la columna por el valor de la UF en pesos (valor al 16/10/2023 => 36.265,46). Dado que se pide el precio de edificar con construcciones de 1 piso, el paso final solo implica multiplicar por la columna que indica el área.

In [ ]:
distritos_urbanos['Precio']=36265.46*distritos_urbanos['Area']*distritos_urbanos['1_UF_M2']

Revisamos...

In [ ]:
distritos_urbanos.head(10)

Finalmente realizamos el plot, utilizando esta nueva columna para la escala de colores

In [ ]:
distritos_urbanos.plot(figsize=(10,10),column='Precio', legend=True)

## Ejemplo 2: construir un ranking de los 5 distritos censales más poblados, que se encuentren a menos de 100 metros de una estación de metro.

### Cargamos el shapefile de estaciones de metro y revisamos sus datos

In [ ]:
estaciones_metro = gpd.read_file('data/Metro 2020/Estaciones_2020/Estaciones_2020.shp')

In [ ]:
estaciones_metro.head()

In [ ]:
estaciones_metro.plot(figsize=(10,10))

### Como siempre, revisamos la proyección antes de aplicar operaciones geométricas

In [ ]:
estaciones_metro.crs

### Para delimitar un área de 100 metros a la redonda de las estaciones, creamos un _buffer_ de 100 metros, el cual posteriormente intersectaremos con los distritos.

In [ ]:
buffer_metro = estaciones_metro.copy()
geometria_buffer = buffer_metro.buffer(100)
geometria_buffer.head()

In [ ]:
geometria_buffer.iloc[0]

In [ ]:
buffer_metro.geometry = geometria_buffer
buffer_metro.head()

### A continuación, intersectamos los distritos con el buffer, esta vez utilizando `sjoin` en vez de `overlay`, ya que no queremos generar nueva geometría, solo filtrar distritos

In [ ]:
distritos_metro = gpd.sjoin(distritos, buffer_metro, predicate='intersects')

¿Qué implica el resultado de la siguiente instrucción?

In [ ]:
distritos_metro['CODIGO'].value_counts().head()

Corregimos el problema

In [ ]:
distritos_metro = distritos_metro.drop_duplicates(subset=['CODIGO'])
distritos_metro['CODIGO'].value_counts().head()

In [ ]:
distritos_metro.describe()

### Finalmente, para conocer la poblacion por distrito, tan solo se tiene que multiplicar la densidad por el área (no es relevante si están o no en la misma escala, ya que solo queremos un ranking)

In [ ]:
poblacion_distrito = distritos_metro['Area']*distritos_metro['Densidad']
distritos_metro['Poblacion'] = poblacion_distrito
distritos_metro.head()

In [ ]:
distritos_metro_ranking = distritos_metro.sort_values(by=['Poblacion'], ascending=False)
distritos_metro_ranking.head()

In [ ]:
ax = distritos_metro.plot(figsize = (10,10), column = 'Poblacion', edgecolor = 'black', linewidth = 0.5,legend = True)
ax1 = buffer_metro.plot(ax=ax, alpha = 0.5, color = 'red')